In [694]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance



In [695]:
df = pd.read_csv('responses.csv')
# df = df[df['QRELSING']==2]

/var/folders/vp/c5kyxyds6jq7j8_tb4ggf_dc0000gn/T/ipykernel_953/4126778955.py:1: DtypeWarning: Columns (47,48,73,74,75,76,77,78,79,80,81,82,83,84,87,91,174,175,209,245) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('responses.csv')


In [696]:
df

,COUNTRY,QRID,weight,QMLangRec,Q2,Q3rec_1,Q3rec_2,Q3rec_3,Q3rec_4,Q3rec_5,...,Q86b_19,Q86b_20,Q86b_21,Q86b_97,Q86b_98,Q86b_99,Q87,QIV7,REGION,Urban
0,1,23,0.776012,2,6,1,2,2,2,2,...,,,,,,,99,1,1,2
1,1,30,0.555058,2,2,1,2,2,2,2,...,,,,,,,99,,2,2
2,1,35,0.238959,2,2,1,2,2,2,2,...,,,,,,,2,,2,2
3,1,37,0.390951,2,1,1,1,2,2,1,...,,,,,,,2,1,5,1
4,1,67,1.553954,2,2,2,1,2,2,2,...,,,,,,,2,,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,1,34925,0.473438,1,2,1,2,2,2,2,...,,,,,,,2,,2,2
29995,1,34926,1.151329,2,6,2,2,2,2,2,...,,,,,,,2,,6,2
29996,1,34927,0.411609,1,1,1,2,2,2,2,...,2,2,2,2,2,2,1,1,2,1
29997,1,34928,0.252903,2,1,1,2,2,2,2,...,,,,,,,2,1,2,2


In [697]:
# get log probablity of all possible responses for Q6a
def compute_log_probabilities(df, column_name):
    # Step 1: Count occurrences of each unique response
    response_counts = df[column_name].value_counts()

    # Step 2: Convert counts to probabilities
    total_responses = len(df[column_name])
    response_probabilities = response_counts / total_responses

    # Step 3: Compute log probabilities
    log_probabilities = np.log(response_probabilities)

    return log_probabilities

log_probabilities = compute_log_probabilities(df, 'Q9')

In [698]:
log_probabilities

1    -0.219201
2    -1.835469
3    -3.963283
98   -4.531267
4    -5.046229
99   -6.812412
Name: Q9, dtype: float64

In [699]:
import ast
def string_to_series(data_string):
    """
    Converts a string representation of a dictionary to a pandas Series.

    Args:
    data_string (str): The string representation of the dictionary.

    Returns:
    pd.Series: A pandas Series with the dictionary data.
    """
    # Convert the string into a Python dictionary
    data_dict = ast.literal_eval(data_string)
    
    # Convert the dictionary into a pandas Series
    series = pd.Series(data_dict)
    
    return series
inp = """   
{'1': -0.3602273166179657, '2': -1.914914846420288, '3': -2.977414846420288, '4': -3.727414846420288}

"""


In [700]:
log_probabilities = log_probabilities.sort_index()
log_probabilities

1    -0.219201
2    -1.835469
3    -3.963283
4    -5.046229
98   -4.531267
99   -6.812412
Name: Q9, dtype: float64

In [701]:
model_probabilities = string_to_series(inp)
human_probabilities = log_probabilities
dist1 = model_probabilities
dist2 = human_probabilities
answer_choices = [1, 2, 3, 4]

# Remove refusal options from distributions
dist1 = dist1[dist1.index.isin(answer_choices)]
dist2 = dist2[dist2.index.isin(answer_choices)]

# Map the ordinal answer choices to integers
dist1 = dist1.reindex(answer_choices, fill_value=0)
dist2 = dist2.reindex(answer_choices, fill_value=0)

# Convert log probabilities to probabilities
prob_dist1 = np.exp(dist1) / np.sum(np.exp(dist1))
prob_dist2 = np.exp(dist2) / np.sum(np.exp(dist2))

# Compute CDFs
cdf_dist1 = np.cumsum(prob_dist1)
cdf_dist2 = np.cumsum(prob_dist2)

# Compute the Wasserstein distance
wasserstein_dist = wasserstein_distance(cdf_dist1, cdf_dist2)
wasserstein_dist

0.3201388186081031

In [702]:
def normalize_non_refusal(series):
    """
    Normalize the probabilities for non-refusal options in a pandas Series of log probabilities.
    
    Parameters:
    - series (pd.Series): Series of log probabilities with answer choices as index.
    
    Returns:
    - pd.Series: Series of normalized probabilities for non-refusal options.
    """
    # Exponentiate log probabilities
    exp_probs = np.exp(series)
    
    # Separate refusal options (98 and 99) and non-refusal options
    refusal_probs = exp_probs[series.index.isin(['98', '99','5'])]
    non_refusal_probs = exp_probs[~series.index.isin(['98', '99','5'])]
    
    # Normalize non-refusal probabilities
    total_non_refusal = np.sum(non_refusal_probs)
    if total_non_refusal > 0:
        normalized_non_refusal_probs = non_refusal_probs / total_non_refusal
    else:
        # If no non-refusal options, set all to zero
        normalized_non_refusal_probs = np.zeros_like(non_refusal_probs)
    return normalized_non_refusal_probs


In [703]:
model_probabilities = string_to_series(inp)
model_probabilities = model_probabilities.sort_index()
#convert model_probabilities key to float
model_probabilities = model_probabilities.astype(float)
model_probabilities

1   -0.360227
2   -1.914915
3   -2.977415
4   -3.727415
dtype: float64

In [704]:
model_non_refusal = normalize_non_refusal(model_probabilities)
model_non_refusal

1    0.758294
2    0.160194
3    0.055361
4    0.026151
dtype: float64

In [705]:
human_probabilities = human_probabilities.astype(float)
human_non_refusal = normalize_non_refusal(human_probabilities)
human_non_refusal

1     0.803160
2     0.159539
3     0.019001
4     0.006434
98    0.010767
99    0.001100
Name: Q9, dtype: float64

In [706]:
human_probabilities.index = human_probabilities.index.astype(str)
human_non_refusal = normalize_non_refusal(human_probabilities)
human_non_refusal

1    0.812806
2    0.161455
3    0.019229
4    0.006511
Name: Q9, dtype: float64

In [707]:
model_non_refusal

1    0.758294
2    0.160194
3    0.055361
4    0.026151
dtype: float64

In [708]:
def kl_div(p, q):
    # Add a small constant to avoid log(0)
    epsilon = 1e-10
    p = p.fillna(0) + epsilon
    q = q.fillna(0) + epsilon
    
    return np.sum(p * np.log(p / q))
def js_divergence(p, q):
    # Normalize to ensure distributions sum to 1
    p = p / p.sum()
    q = q / q.sum()
    
    m = 0.5 * (p + q)
    return 0.5 * (kl_div(p, m).sum() + kl_div(q, m).sum())
js_divergence(model_non_refusal, human_non_refusal)

0.008202171303669285

In [709]:
wasserstein_distance(model_non_refusal, human_non_refusal)

0.027886359896042438